In [1]:
import pandas as pd
import numpy as np
import random
import os
import csv
import json
import struct

1\. **Text files**

Perform the following operations on plain `txt` files:

+ create a list of integrer numbers and then save it to a text file named `data_int.txt`. Run the `cat` command to print the content of the file.
+ create a matrix of 5x5 floats and then save it to a text file named `data_float.txt`. Use the `cat` command to print the content of the file.
+ load the `txt` file of the previous point and convert it to a `csv` file by hand.

In [2]:
def ex_1_1():
    a=random.randint(1,10)
    lista=[]
    for values in range(a):
        lista.append(random.randint(1,10))
    file = open('data_int.txt', 'w')
    file.write(str(lista))
    file.close()
    os.system('cat data_int.txt')
    
def ex_1_2():
    a=np.matrix('1.1 2.1 3.1 4.1 5.1;1.1 2.1 3.1 4.1 5.1;1.1 2.1 3.1 4.1 5.1;1.1 2.1 3.1 4.1 5.1;1.1 2.1 3.1 4.1 5.1')
    np.savetxt("data_float.txt", a)
    os.system('cat data_float.txt')

def load_txt():  
    text=""
    with open (r'data_float.txt') as read_file:#Loading part
        for line in read_file:
            text=text+line+"\n"
            
    with open('data1_float.csv', 'w') as data1:
        data1.write(text)
        #read_file.to_csv (r'data1_float.csv') 
        

#ex_1_2()
load_txt()

FileNotFoundError: [Errno 2] No such file or directory: 'data_float.txt'

2\. **JSON files**

Load the file *user_data.json*, which can be found at:

- https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json

and filter the data by the "CreditCardType" when it equals to "American Express". Than save the data to a new CSV file.

In [ ]:
#!wget https://www.dropbox.com/s/sz5klcdpckc39hd/user_data.json
    
#input_dict = json.loads('user_data.json')
with open('user_data.json', 'r') as f:
    data = json.load(f)
output_dict = [x for x in data if x['CreditCardType'] == 'American Express']
output_json = json.dumps(output_dict)

df = pd.read_json (output_json)
df.to_csv (r'user1.csv')

3\. **CSV files with Pandas**

Load the file from this url:

- https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv

with Pandas. 

+ explore and print the DataFrame
+ calculate, using `groupby()`, the average value of each feature, separately for each class
+ save the file in a JSON format.

In [ ]:
#!wget https://www.dropbox.com/s/kgshemfgk22iy79/mushrooms_categorized.csv
df = pd.read_csv(r'mushrooms_categorized.csv')
a=df.groupby(by="class").mean()
a.to_json(r'mushrooms.json')

4\. **Reading the credit card numbers**

Get the binary file named *credit_card.dat* from this address:

- https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

and convert the data into the real credit card number, knowing that:
- each line corresponds to a credit card number, which consists of 16 characters (which are numbers in the 0-9 range) divided in 4 blocks, with a whitespace between each block
- each character is written using a 6 bit binary representation (including the whitespace)
- the final 4 bits of each line are a padding used to determine the end of the line, and can be ignored

*Hint*: convert the binary numbers to the decimal representation first, and then use the `chr()` function to convert the latter to a char

In [ ]:
#!wget https://www.dropbox.com/s/8m0syw2tkul3dap/credit_card.dat

datContent = [i.strip().split() for i in open("credit_card.dat").readlines()]


with open("cc.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(datContent)
    
block=[]

with open("cc.csv", "r") as f:
    for lines in f:
        if len(lines)>4:
            block.append(lines[0:24])
            block.append(lines[30:54])
            block.append(lines[60:84])
            block.append(lines[90:114])
            value=""
            for blocks in block:
                for i in range(4):
                    value=value+str(int(blocks[6*i+2:6*i+6],2))
            block=[]
            print(value)

5\. **Write data to a binary file**

a) Start from the `data/data_000637.txt` file that we have used during the previous lectures, and convert it to a binary file according to the format defined below:

In [ ]:
from IPython.display import Image
Image("data_format.png")

*Hints*:
- Read the first 10 lines using Pandas
- Iterate over the DataFrame rows
- For every row, ``pack'' the values (features) into a single 64-bit word, according to the format specified above. Use bit-wise shifts and operators to do so.
- Write each 64-bit word to a binary file. You can use `struct` in this way:
```
binary_file.write( struct.pack('<q', word) )
```
where `word` is the 64-bit word.
- Close the file after completing the loop.

b) Check that the binary file is correctly written by reading it with the code used in the lecture `06_OSEMN.ipynb`, and verify that the content of the `txt` and binary files is consistent.

c) What is the difference of the size on disk between equivalent `txt` and binary files?

In [ ]:
file_name = "data_000637.txt" 
A=10
df_c = pd.read_csv(file_name)

#FASE DI SCRITTURA
with open('data_000637_to_binary.dat', 'wb') as binary_file:    
    
    for index, row in df_c.iterrows():
        word = 0
        word = (row['HEAD'] << 62) | word
        word = (row['FPGA'] << 58) | word
        word = (row['TDC_CHANNEL'] << 49) | word
        word = (row['ORBIT_CNT'] << 17) | word
        word = (row['BX_COUNTER'] << 5 ) | word
        word = (row['TDC_MEAS'] << 0 ) | word
        binary_file.write(struct.pack('<q', word))
        
data = {}
columns = ['HEAD', 'FPGA', 'CHANNEL', 'ORBIT_CNT', 'BX_CNT', 'TDC_MEAS']
df = pd.DataFrame({}, columns=columns)

#FASE DI LETTURA PER IL CHECK
with open('data_000637_to_binary.dat', 'rb') as file:
    file_content = file.read()
    cnt = 0
    word_size = 8
    
    for i in range(0, len(file_content), word_size):
        
        cnt= cnt+1
        word = struct.unpack('<q', file_content[i : i + word_size])[0] # get an 8-byte word
        head     = (word >> 62) & 0x3
        fpga     = (word >> 58) & 0xF
        tdc_chan = (word >> 49) & 0x1FF
        orb_cnt  = (word >> 17) & 0xFFFFFFFF
        bx       = (word >> 5 ) & 0xFFF
        tdc_meas = (word >> 0 ) & 0x1F
        entry = {'HEAD' : head, 'FPGA' : fpga, 'CHANNEL' : tdc_chan, 'ORBIT_CNT' : orb_cnt, 'BX_CNT' : bx, 'TDC_MEAS' : tdc_meas}
        df = df.append(entry, ignore_index=True)
        if cnt > 10: 
            break
      
print(df)
print(df_c)

#La memorizzazione del binario è approssimativamente 1/3 del txt originale.